Primero cargaremos todo las librerias que usaremos y tambien el archivo.csv de canciones

In [2]:
import pandas as pd
import numpy as np
import librosa
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

songs= pd.read_csv('../new_spotify_songs.csv')
songs.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0zEKOcdfzNkvW1OFnazAQO,sugar honey ice & tea - Edit,Bring Me The Horizon,Count down the thunder I think we're too close...,55,2442fVHcs5gEuDXgRHespx,sugar honey ice & tea (Edit),2019-07-26,Rock Hard,37i9dQZF1DWWJOmJ7nRx0C,...,-3.765,0,0.0513,0.000075,0.003900,0.2470,0.685,147.171,215712,en
1,0ZfM5XfJTtFPhOxAERRnNY,The Look,Roxette,"One, two, three, four Walkin' like a man Hitti...",72,1iI5YZkqNUV7VmrEi4uOP9,Look Sharp! (2009 Version),1988-10-19,Electropop 80/90s,3FBD3d5zl74LaoBMVC4IXq,...,-5.363,1,0.0413,0.053100,0.000008,0.0522,0.541,95.011,237320,en
2,0zG6VemmYlMhAK5MNvBeP7,No Sleep,Bougenvilla,We don't get no sleep We don't get no sleep If...,0,6k2LCRFumiFy6OlIGzLtOx,No Sleep,2017-04-03,🔊BASSBOOSTED🔊⚡ELECTRO HOUSE⚡🔥EDM CAR MUSIC2018...,4GSiiL8tcMgvoV7K1IADb8,...,-3.586,0,0.0401,0.016300,0.143000,0.3070,0.199,127.973,209664,en
3,0ZGLuduCPjgWY1n85ykgMe,Back In the Saddle,Aerosmith,I'm back I'm back in the saddle again I'm back...,60,5Uv5LmSKTT9okGkr3l9MjR,Rocks,1976-05-14,Supernatural Classic Rock,6oghIlByD49KFGNmNU8GSH,...,-9.661,1,0.0389,0.005780,0.009450,0.0945,0.762,122.201,280360,en
4,0ZgTez1pojM5aCxsV0hrC9,No Guarantee - Remix Version,Chico DeBarge,Got a one room apartment To a little tiny spac...,34,0AHbQKIEj4X5ghszLRhnLL,Long Time No See,1997-11-18,90s R&B - The BET Planet Groove/Midnight Love Mix,7LjRWGUdOmBdPG1W1bga4i,...,-5.789,0,0.1190,0.064600,0.000000,0.2610,0.623,127.885,244467,en


Cargamos el csv de vector caracteristico de las canciones

In [3]:
features = pd.read_csv('../new_features.csv')
features.head()

,track_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0zEKOcdfzNkvW1OFnazAQO,-0.098841,-0.116778,-0.005992,-0.076890,0.167060,-0.153227,-0.127395,0.062021,-0.143093,...,-0.134901,-0.126881,-0.211531,-0.137909,0.010951,0.330660,0.048009,0.161574,0.138966,0.203755
1,0ZfM5XfJTtFPhOxAERRnNY,-0.329168,-0.208267,-0.242830,0.158825,-0.002144,0.104842,0.214938,0.311463,0.014895,...,-0.117848,0.411027,0.117565,0.494875,0.159988,-0.554037,-0.252416,-0.382986,-0.372454,-0.487503
2,0zG6VemmYlMhAK5MNvBeP7,0.505574,0.732437,-0.009727,-0.097279,-0.436935,-0.585198,-0.322022,-0.157966,-0.272551,...,-0.062521,-0.071936,-0.313199,0.065619,0.130820,-0.027184,0.255665,0.166922,0.731590,0.160565
3,0ZGLuduCPjgWY1n85ykgMe,-0.155610,0.135038,-0.230663,-0.338267,-0.108828,0.285991,0.326603,0.131136,-0.333797,...,-0.213955,-0.266517,-0.115268,-0.004639,-0.237066,-0.114990,-0.064895,0.224829,-0.642008,0.316891
4,0ZgTez1pojM5aCxsV0hrC9,0.157820,-0.082984,-0.005262,-0.172845,-0.112333,-0.223888,-0.055531,0.009561,-0.095176,...,-0.165788,-0.390585,0.204284,0.097674,0.198702,0.039575,-0.120027,-0.157962,-0.197737,0.215317


Verficamos que tenemos lo necesario par auna busqueda textual con pandas solo extraemos las columnas "track_name", "track_artist", "lyrics","track_id". Esto no es necesario, pero se implemento con KNN.

In [4]:
essential = songs[["track_name", "track_artist", "lyrics","track_id"]]
essential.head()

,track_name,track_artist,lyrics,track_id
0,sugar honey ice & tea - Edit,Bring Me The Horizon,Count down the thunder I think we're too close...,0zEKOcdfzNkvW1OFnazAQO
1,The Look,Roxette,"One, two, three, four Walkin' like a man Hitti...",0ZfM5XfJTtFPhOxAERRnNY
2,No Sleep,Bougenvilla,We don't get no sleep We don't get no sleep If...,0zG6VemmYlMhAK5MNvBeP7
3,Back In the Saddle,Aerosmith,I'm back I'm back in the saddle again I'm back...,0ZGLuduCPjgWY1n85ykgMe
4,No Guarantee - Remix Version,Chico DeBarge,Got a one room apartment To a little tiny spac...,0ZgTez1pojM5aCxsV0hrC9


Limpieza de datos nulos y duplicados

In [5]:
essential = essential.dropna()
essential = essential.drop_duplicates()
essential.shape
essential.head()

,track_name,track_artist,lyrics,track_id
0,sugar honey ice & tea - Edit,Bring Me The Horizon,Count down the thunder I think we're too close...,0zEKOcdfzNkvW1OFnazAQO
1,The Look,Roxette,"One, two, three, four Walkin' like a man Hitti...",0ZfM5XfJTtFPhOxAERRnNY
2,No Sleep,Bougenvilla,We don't get no sleep We don't get no sleep If...,0zG6VemmYlMhAK5MNvBeP7
3,Back In the Saddle,Aerosmith,I'm back I'm back in the saddle again I'm back...,0ZGLuduCPjgWY1n85ykgMe
4,No Guarantee - Remix Version,Chico DeBarge,Got a one room apartment To a little tiny spac...,0ZgTez1pojM5aCxsV0hrC9


Obtenemos el tfidvectorizer de nuestra dataframe

In [6]:
vectorizer = TfidfVectorizer()
vectorizer
X = vectorizer.fit_transform(essential["lyrics"])
X

<15110x92553 sparse matrix of type '<class 'numpy.float64'>'
	with 1758291 stored elements in Compressed Sparse Row format>

KNN secuencial de busqueda textual implementada

In [7]:
def KNN_secuencial(query, k):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    top_k_indices = results.argsort()[-k:][::-1]
    top_k_songs = essential.iloc[top_k_indices][["track_name", "track_artist"]]
    return top_k_songs
query = "I'm a rebel just for kicks, now"
k=5
print(KNN_secuencial(query, k))




                                 track_name       track_artist
13432                         Feel It Still  Portugal. The Man
14952                         Feel It Still  Portugal. The Man
6383   It's All on U (feat. Liam O'Donnell)           ILLENIUM
12028           Rebel Rebel - 1999 Remaster        David Bowie
13358           Rebel Rebel - 2014 Remaster        David Bowie


Tambien se implemento una variacion con heap

In [8]:
import heapq
def KNN_secuencial_heap(query, k):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    top_k_indices = heapq.nlargest(k, range(len(results)), results.take)
    top_k_songs = essential.iloc[top_k_indices][["track_name", "track_artist"]]
    return top_k_songs

print(KNN_secuencial_heap(query, k))

                                 track_name       track_artist
13432                         Feel It Still  Portugal. The Man
14952                         Feel It Still  Portugal. The Man
6383   It's All on U (feat. Liam O'Donnell)           ILLENIUM
3154            Rebel Rebel - 2016 Remaster        David Bowie
10240           Rebel Rebel - 1997 Remaster        David Bowie


Se implemento una busqueda textual con knn_range_search, no es necesario

In [9]:
radius = 0.2
def KNN_range_Search(query, radius):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X, query_vec).reshape((-1,))
    indices = np.where(results >= radius)[0]
    songs = essential.iloc[indices][["track_name", "track_artist"]]
    return songs

print(KNN_range_Search(query, radius))

                                 track_name       track_artist
3154            Rebel Rebel - 2016 Remaster        David Bowie
6383   It's All on U (feat. Liam O'Donnell)           ILLENIUM
7605                              Pumped Up          Klingande
9151                                McQueen          TrifeDrew
10240           Rebel Rebel - 1997 Remaster        David Bowie
12028           Rebel Rebel - 1999 Remaster        David Bowie
13037                             Pumped Up          Klingande
13358           Rebel Rebel - 2014 Remaster        David Bowie
13424              Rebel Rebel - Remastered        David Bowie
13432                         Feel It Still  Portugal. The Man
14622                         Teenage Kicks     The Undertones
14952                         Feel It Still  Portugal. The Man


In [10]:
first_value = songs.loc[0]
print(first_value)

track_id                                               0zEKOcdfzNkvW1OFnazAQO
track_name                                       sugar honey ice & tea - Edit
track_artist                                             Bring Me The Horizon
lyrics                      Count down the thunder I think we're too close...
track_popularity                                                           55
track_album_id                                         2442fVHcs5gEuDXgRHespx
track_album_name                                 sugar honey ice & tea (Edit)
track_album_release_date                                           2019-07-26
playlist_name                                                       Rock Hard
playlist_id                                            37i9dQZF1DWWJOmJ7nRx0C
playlist_genre                                                           rock
playlist_subgenre                                                   hard rock
danceability                                                    

Esta parte es lo que se pidio implementar, extraemos las mismas columnas importantes "track_name", "track_artist", "lyrics","track_id" para verificar si resulto con existe vemos las primeras filas con .head() de nuestra dataframe.

In [11]:
essential_2 = songs[['track_id', 'track_name', 'track_artist', 'lyrics']]
essential_2.head()

,track_id,track_name,track_artist,lyrics
0,0zEKOcdfzNkvW1OFnazAQO,sugar honey ice & tea - Edit,Bring Me The Horizon,Count down the thunder I think we're too close...
1,0ZfM5XfJTtFPhOxAERRnNY,The Look,Roxette,"One, two, three, four Walkin' like a man Hitti..."
2,0zG6VemmYlMhAK5MNvBeP7,No Sleep,Bougenvilla,We don't get no sleep We don't get no sleep If...
3,0ZGLuduCPjgWY1n85ykgMe,Back In the Saddle,Aerosmith,I'm back I'm back in the saddle again I'm back...
4,0ZgTez1pojM5aCxsV0hrC9,No Guarantee - Remix Version,Chico DeBarge,Got a one room apartment To a little tiny spac...


Limpieza de datos:

- **Eliminamos Datos nulos**

- **Eliminamos Datos duplicados**

Verificamos la cantidad de elementos se redujo o se mantuvo

In [12]:
essential_2 = essential_2.dropna()
essential_2 = essential_2.drop_duplicates()
essential_2.shape


(15110, 4)

Hacemos el merge entre nuestra dataframe extraida essential2 y nuestro csv con el vector caracteristico cargado anteriormente.


In [13]:
combined_data = pd.merge(essential_2, features, on='track_id')
combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0zEKOcdfzNkvW1OFnazAQO,sugar honey ice & tea - Edit,Bring Me The Horizon,Count down the thunder I think we're too close...,-0.098841,-0.116778,-0.005992,-0.076890,0.167060,-0.153227,...,-0.134901,-0.126881,-0.211531,-0.137909,0.010951,0.330660,0.048009,0.161574,0.138966,0.203755
1,0ZfM5XfJTtFPhOxAERRnNY,The Look,Roxette,"One, two, three, four Walkin' like a man Hitti...",-0.329168,-0.208267,-0.242830,0.158825,-0.002144,0.104842,...,-0.117848,0.411027,0.117565,0.494875,0.159988,-0.554037,-0.252416,-0.382986,-0.372454,-0.487503
2,0zG6VemmYlMhAK5MNvBeP7,No Sleep,Bougenvilla,We don't get no sleep We don't get no sleep If...,0.505574,0.732437,-0.009727,-0.097279,-0.436935,-0.585198,...,-0.062521,-0.071936,-0.313199,0.065619,0.130820,-0.027184,0.255665,0.166922,0.731590,0.160565
3,0ZGLuduCPjgWY1n85ykgMe,Back In the Saddle,Aerosmith,I'm back I'm back in the saddle again I'm back...,-0.155610,0.135038,-0.230663,-0.338267,-0.108828,0.285991,...,-0.213955,-0.266517,-0.115268,-0.004639,-0.237066,-0.114990,-0.064895,0.224829,-0.642008,0.316891
4,0ZgTez1pojM5aCxsV0hrC9,No Guarantee - Remix Version,Chico DeBarge,Got a one room apartment To a little tiny spac...,0.157820,-0.082984,-0.005262,-0.172845,-0.112333,-0.223888,...,-0.165788,-0.390585,0.204284,0.097674,0.198702,0.039575,-0.120027,-0.157962,-0.197737,0.215317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15105,7k4t7uLgtOxPwTpFmtJNTY,Tusa,KAROL G,NA ¿Qué pasa contigo? Dímelo Rrr!O-O-Ovy on th...,-0.602929,-0.277219,-0.493499,0.094192,0.304649,-0.014173,...,-0.193526,-0.226324,0.042981,0.040540,0.163899,0.242983,0.437682,0.234670,-0.081395,-1.041992
15106,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,I know that I am runnin' out of time I want it...,-0.451623,-0.234071,-0.159290,-0.161218,-0.095926,-0.051664,...,-0.119346,-0.139924,-0.272772,-0.386242,0.218397,0.275528,0.536367,0.256531,0.399271,-0.691504
15107,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,"(Chuck D: Here we go again) Well, I'm out to h...",0.656685,0.437153,0.526300,0.198158,0.008960,-0.278656,...,-0.006828,-0.205763,0.011333,0.094089,0.197834,0.163011,0.098527,-0.196720,-0.258613,0.252062
15108,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,"Uh, Next Selection You've been waiting so long...",0.884435,0.957433,0.266550,-0.403613,-0.411181,0.208359,...,0.028934,0.065975,0.331138,0.341994,0.015794,-0.213025,-0.156314,-0.068632,-0.226346,-0.478887


Limipieza de datos luego del merge:

- **Eliminamos Datos nulos**

- **Eliminamos Datos duplicados**



In [14]:
normalized_combined_data = combined_data.dropna() #Elimino la tabla con valores nulos
normalized_combined_data = combined_data.drop_duplicates() #Elimino los duplicados

normalized_combined_data.shape

(15110, 364)

In [15]:

# Seleccionar solo las columnas numéricas
features_to_normalize = combined_data.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
features_to_normalize.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,-0.098841,-0.116778,-0.005992,-0.076890,0.167060,-0.153227,-0.127395,0.062021,-0.143093,0.101124,...,-0.134901,-0.126881,-0.211531,-0.137909,0.010951,0.330660,0.048009,0.161574,0.138966,0.203755
1,-0.329168,-0.208267,-0.242830,0.158825,-0.002144,0.104842,0.214938,0.311463,0.014895,-0.119139,...,-0.117848,0.411027,0.117565,0.494875,0.159988,-0.554037,-0.252416,-0.382986,-0.372454,-0.487503
2,0.505574,0.732437,-0.009727,-0.097279,-0.436935,-0.585198,-0.322022,-0.157966,-0.272551,-0.103481,...,-0.062521,-0.071936,-0.313199,0.065619,0.130820,-0.027184,0.255665,0.166922,0.731590,0.160565
3,-0.155610,0.135038,-0.230663,-0.338267,-0.108828,0.285991,0.326603,0.131136,-0.333797,0.013289,...,-0.213955,-0.266517,-0.115268,-0.004639,-0.237066,-0.114990,-0.064895,0.224829,-0.642008,0.316891
4,0.157820,-0.082984,-0.005262,-0.172845,-0.112333,-0.223888,-0.055531,0.009561,-0.095176,0.091638,...,-0.165788,-0.390585,0.204284,0.097674,0.198702,0.039575,-0.120027,-0.157962,-0.197737,0.215317


Limpieza de datos a esta nueva dataframe:

- **Verificar si hay valores NaN en el DataFrame**

- **Mostrar las filas con al menos un valor NaN**

- **Mostrar el número total de filas con al menos un valor NaN**

In [16]:
nan_check = normalized_combined_data.isna().any(axis=1)

rows_with_nan = normalized_combined_data[nan_check]
print(f"Filas con al menos un valor NaN:\n{rows_with_nan}")

total_rows_with_nan = rows_with_nan.shape[0]
print(f"Número total de filas con al menos un valor NaN: {total_rows_with_nan}")


Filas con al menos un valor NaN:
Empty DataFrame
Columns: [track_id, track_name, track_artist, lyrics, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15, f16, f17, f18, f19, f20, f21, f22, f23, f24, f25, f26, f27, f28, f29, f30, f31, f32, f33, f34, f35, f36, f37, f38, f39, f40, f41, f42, f43, f44, f45, f46, f47, f48, f49, f50, f51, f52, f53, f54, f55, f56, f57, f58, f59, f60, f61, f62, f63, f64, f65, f66, f67, f68, f69, f70, f71, f72, f73, f74, f75, f76, f77, f78, f79, f80, f81, f82, f83, f84, f85, f86, f87, f88, f89, f90, f91, f92, f93, f94, f95, f96, ...]
Index: []

[0 rows x 364 columns]
Número total de filas con al menos un valor NaN: 0


Verficamos nuestra dataframe limpia.

In [17]:
normalized_combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0zEKOcdfzNkvW1OFnazAQO,sugar honey ice & tea - Edit,Bring Me The Horizon,Count down the thunder I think we're too close...,-0.098841,-0.116778,-0.005992,-0.076890,0.167060,-0.153227,...,-0.134901,-0.126881,-0.211531,-0.137909,0.010951,0.330660,0.048009,0.161574,0.138966,0.203755
1,0ZfM5XfJTtFPhOxAERRnNY,The Look,Roxette,"One, two, three, four Walkin' like a man Hitti...",-0.329168,-0.208267,-0.242830,0.158825,-0.002144,0.104842,...,-0.117848,0.411027,0.117565,0.494875,0.159988,-0.554037,-0.252416,-0.382986,-0.372454,-0.487503
2,0zG6VemmYlMhAK5MNvBeP7,No Sleep,Bougenvilla,We don't get no sleep We don't get no sleep If...,0.505574,0.732437,-0.009727,-0.097279,-0.436935,-0.585198,...,-0.062521,-0.071936,-0.313199,0.065619,0.130820,-0.027184,0.255665,0.166922,0.731590,0.160565
3,0ZGLuduCPjgWY1n85ykgMe,Back In the Saddle,Aerosmith,I'm back I'm back in the saddle again I'm back...,-0.155610,0.135038,-0.230663,-0.338267,-0.108828,0.285991,...,-0.213955,-0.266517,-0.115268,-0.004639,-0.237066,-0.114990,-0.064895,0.224829,-0.642008,0.316891
4,0ZgTez1pojM5aCxsV0hrC9,No Guarantee - Remix Version,Chico DeBarge,Got a one room apartment To a little tiny spac...,0.157820,-0.082984,-0.005262,-0.172845,-0.112333,-0.223888,...,-0.165788,-0.390585,0.204284,0.097674,0.198702,0.039575,-0.120027,-0.157962,-0.197737,0.215317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15105,7k4t7uLgtOxPwTpFmtJNTY,Tusa,KAROL G,NA ¿Qué pasa contigo? Dímelo Rrr!O-O-Ovy on th...,-0.602929,-0.277219,-0.493499,0.094192,0.304649,-0.014173,...,-0.193526,-0.226324,0.042981,0.040540,0.163899,0.242983,0.437682,0.234670,-0.081395,-1.041992
15106,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,I know that I am runnin' out of time I want it...,-0.451623,-0.234071,-0.159290,-0.161218,-0.095926,-0.051664,...,-0.119346,-0.139924,-0.272772,-0.386242,0.218397,0.275528,0.536367,0.256531,0.399271,-0.691504
15107,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,"(Chuck D: Here we go again) Well, I'm out to h...",0.656685,0.437153,0.526300,0.198158,0.008960,-0.278656,...,-0.006828,-0.205763,0.011333,0.094089,0.197834,0.163011,0.098527,-0.196720,-0.258613,0.252062
15108,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,"Uh, Next Selection You've been waiting so long...",0.884435,0.957433,0.266550,-0.403613,-0.411181,0.208359,...,0.028934,0.065975,0.331138,0.341994,0.015794,-0.213025,-0.156314,-0.068632,-0.226346,-0.478887


Esta parte es necesaria para la query de modo que crearemos un vector caracteristico para esta consulta, necesariamente la misma cantidad de caracteristicas que de las canciones.

## Funcion de extraccion de caracteristicas del audio

In [18]:
scaler = preprocessing.StandardScaler()
def extract_features(file_path):
    # Cargar el archivo de audio
    y, sr = librosa.load(file_path)

    # Extraer características
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    
    #Estandarizamos
    chroma_stft = scaler.fit_transform(chroma_stft.T)
    chroma_stft = chroma_stft.T
    # Dividir en segmentos uniformes
    num_segments = 30
    segment_len = chroma_stft.shape[1] // num_segments

    # Tomar la media de cada segmento
    chroma_stft_uniform = np.mean(chroma_stft[:, :num_segments * segment_len].reshape(12, -1, segment_len), axis=2)
    # Concatenar todas las características en un solo vector
    features = chroma_stft_uniform.flatten()
    
    return features

### Consulta 1

In [19]:
audio_path="feel.mp3"
scaled_audio_features=pd.DataFrame(extract_features(audio_path)).transpose()
scaled_audio_features

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,0.373056,0.107478,0.184831,-0.162873,-0.117238,0.217079,0.075112,0.650628,0.10612,0.021485,...,-0.410744,0.101319,-0.287646,0.274538,0.236566,-0.270106,0.438478,0.154848,0.114364,-0.638817


### Consulta 2

In [20]:
audio_path="query2.wav"
scaled_audio_features2=pd.DataFrame(extract_features(audio_path)).transpose()
scaled_audio_features2

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,1.324466,0.022579,0.309826,0.806057,0.971554,0.526042,0.09857,-0.865484,1.279644,1.091261,...,-0.915616,-1.080968,-1.216198,0.449409,0.800919,0.639257,-0.287134,-0.891652,-1.021489,-0.421016


### Consulta 3

In [21]:
audio_path="cinderella.wav"
scaled_audio_features3=pd.DataFrame(extract_features(audio_path)).transpose()
scaled_audio_features3

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,-1.039753,0.034764,0.102336,-0.440019,-0.426903,0.748025,-0.372351,-0.257574,-0.365496,0.970163,...,0.287135,0.209575,0.009633,0.313954,0.362166,-0.040241,0.262616,-0.077819,0.112796,-0.562266


## MODELO KNN

EL modelo KNN search se implementa con ayudade sklearn que nos proporciona la facilidad con los modelos de un arbol KD conseguir los vecinos mas cercanos

### Sin cola de prioridad

In [22]:
from sklearn.neighbors import NearestNeighbors

def knn_search(query_audio_features, reference_features, k=5, metric='euclidean'):
    # Inicializar el modelo KNN
    knn_model = NearestNeighbors(n_neighbors=k, metric=metric)

    # Extraer las características numéricas para el modelo KNN, 4 primeras columnas del data frame
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
    knn_model.fit(reference_features_numeric)

    # Convertir query_audio_features a un array NumPy
    query_features_numeric = query_audio_features.to_numpy()

    # Utilizar el modelo para encontrar los k vecinos más cercanos para la query del audio
    distances, indices = knn_model.kneighbors(query_features_numeric)

    # Obtener las k canciones más cercanas utilizando los índices
    nearest_songs = reference_features.iloc[indices[0], :4]  # Obtener solo las primeras 4 columnas

    return nearest_songs


### Con cola de prioridad

In [23]:
from queue import PriorityQueue
from sklearn.neighbors import NearestNeighbors

def knn_search_priority_queue(query_audio_features, reference_features, k=5, metric='euclidean'):
    # Initialize the KNN model
    knn_model = NearestNeighbors(n_neighbors=k, metric=metric)

    # Extract the numeric features for the KNN model, excluding the first 4 columns
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
    knn_model.fit(reference_features_numeric)

    # Convert query_audio_features to a NumPy array
    query_features_numeric = query_audio_features.to_numpy()

    # Use the model to find the k nearest neighbors for the audio query
    distances, indices = knn_model.kneighbors(query_features_numeric)

    # Create a priority queue to store the k nearest neighbors
    pq = PriorityQueue()

    # Iterate over the indices and distances to add the neighbors to the priority queue
    for i in range(len(indices[0])):
        index = indices[0][i]
        distance = distances[0][i]
        pq.put((distance, index))

    # Get the k nearest neighbors from the priority queue
    nearest_songs = []
    while not pq.empty():
        distance, index = pq.get()
        nearest_songs.append(reference_features.iloc[index, :4])  # Get only the first 4 columns

    return nearest_songs


## Visto en clase

In [24]:
def knn_search_simple(Q, collection, k):
    result = []

    for _, row in collection.iterrows():
        C = row.iloc[4:].to_numpy()  # Obtener el vector característico desde la quinta columna hasta el final

        # Calcular la distancia para cada fila de la matriz de consulta Q
        distances = [distance(q, C) for q in Q]

        # Obtener el índice del elemento más cercano para cada fila de Q
        indices = np.argsort(distances)

        # Agregar resultados a la lista
        for i in range(len(Q)):
            result.append({
                'track_id': row['track_id'],
                'track_name': row['track_name'],
                'track_artist': row['track_artist'],
                'distance': distances[i],
                'query_row': i
            })

    result.sort(key=lambda x: x['distance'])  # Ordena por distancia en orden ascendente
    return result[:k+1]  # Devuelve los k vecinos más cercanos




def distance(vector1, vector2):
    return np.linalg.norm(vector1 - vector2)

### Con Heap

In [47]:
import heapq
def knn_search_simple_priority_queue(Q, collection, k, distance_function=distance):
    result = []

    for _, row in collection.iterrows():
        C = row.iloc[4:].to_numpy()  # Obtener el vector característico desde la quinta columna hasta el final

        # Calcular la distancia para cada fila de la matriz de consulta Q
        distances = [distance_function(q, C) for q in Q]

        # Crear una cola de prioridad con los k elementos más cercanos
        closest_queue = []
        for i, distance in enumerate(distances):
            if len(closest_queue) < k:
                heapq.heappush(closest_queue, (distance, i))
            else:
                if distance < closest_queue[0][0]:
                    heapq.heappop(closest_queue)
                    heapq.heappush(closest_queue, (distance, i))

        # Agregar resultados a la lista
        for _, i in closest_queue:
            result.append({
                'track_id': row['track_id'],
                'track_name': row['track_name'],
                'track_artist': row['track_artist'],
                'distance': distances[i],
                'query_row': i
            })

    return result[:k]



## MODELO KNN - arboles KD

## Query 1


In [26]:
k_nearest_songs = knn_search(scaled_audio_features, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
12333,6EZDbjz2X5XzgIZCtbsrXh,The Passenger,Siouxsie and the Banshees,I am the passenger And I ride and I ride I rid...
11297,5VU2hajdO59NNjIz9p9dYD,High So High (feat. Marilyn Rylander),South Park Mexican,(Verse 1) I'm a roll '84's til them hoes start...
4973,37zQVgP3aTLKNvluXLB5Ii,Picture Me Rollin',2Pac,"Yeah, clear enough for ya? (alright) Why nigga..."
1357,1lJZSsMoWEbrgaY6CxPMf8,Would I Lie to You? - Remastered Version,Eurythmics,Would I lie to you? (oh yeah) Would I lie to y...
9702,5a251qN4R86hro8UZGuO4E,The Streets - Re-Twist,WC,Yea Turn the music up a little bit Dah-dah-daa...


In [27]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
12333,6EZDbjz2X5XzgIZCtbsrXh,The Passenger,Siouxsie and the Banshees,I am the passenger And I ride and I ride I rid...
11297,5VU2hajdO59NNjIz9p9dYD,High So High (feat. Marilyn Rylander),South Park Mexican,(Verse 1) I'm a roll '84's til them hoes start...
4973,37zQVgP3aTLKNvluXLB5Ii,Picture Me Rollin',2Pac,"Yeah, clear enough for ya? (alright) Why nigga..."
1357,1lJZSsMoWEbrgaY6CxPMf8,Would I Lie to You? - Remastered Version,Eurythmics,Would I lie to you? (oh yeah) Would I lie to y...
9702,5a251qN4R86hro8UZGuO4E,The Streets - Re-Twist,WC,Yea Turn the music up a little bit Dah-dah-daa...


## Query 2

In [28]:
k_nearest_songs = knn_search(scaled_audio_features2, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
4501,2xE16muNCmhZwSOv3i1d1H,Mood Swings,A Boogie Wit da Hoodie,"Wheezy outta here Beast, beast I'm a beast Oh ..."
8496,4OuuTwky6Lupc7zDJNtig6,Uitzicht,Bakermat,Instrumental
10821,5PM96PMKMfD1lLX2lryUsG,Oh Sheila,Ready For The World,"Like they always say, what's good for the goos..."
6693,3VwZqgfrM3xb1usuLprkTu,Cellophane,FKA twigs,Didn't I do it for you? Why don't I do it for ...
9495,54o97glZqjfQambK2yyNCG,Me Liberé,Evaluna Montaner,Ya me liberé de tu falso amor Voy a celebrar q...


In [29]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features2, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
4501,2xE16muNCmhZwSOv3i1d1H,Mood Swings,A Boogie Wit da Hoodie,"Wheezy outta here Beast, beast I'm a beast Oh ..."
8496,4OuuTwky6Lupc7zDJNtig6,Uitzicht,Bakermat,Instrumental
10821,5PM96PMKMfD1lLX2lryUsG,Oh Sheila,Ready For The World,"Like they always say, what's good for the goos..."
6693,3VwZqgfrM3xb1usuLprkTu,Cellophane,FKA twigs,Didn't I do it for you? Why don't I do it for ...
9495,54o97glZqjfQambK2yyNCG,Me Liberé,Evaluna Montaner,Ya me liberé de tu falso amor Voy a celebrar q...


## Query 3

In [30]:
k_nearest_songs = knn_search(scaled_audio_features3, normalized_combined_data, k=10)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
1748,1qhgBzRH0jOzoVZJ2K1hDn,Royal Flush (feat. André 3000 & Raekwon),Big Boi,I am the wrong nigga to cross and the first ni...
1598,1ohXrU5DLEfUlfo4tIYbqJ,Where You Belong,Jay Prince,Yeah this is where you belong This is where yo...
7828,4fsyPCAb55yWiyJVbDJLr5,Falling for You,Tamia,HOOK: Could I be falling for you Is this a fan...
13027,6NJ5Q5QujNwwsst5mdSEfF,Die Happy,DREAMERS,"Yeah, I could die happy for a moment Happy 'ca..."
187,13XHuE00ElL5thSxkaEXxK,Only God Can Judge Me,2Pac,Only God can judge me (that right?) Only God c...
14157,74Yw8aWE60rlSAeT49h9BS,Foe Life,Mack 10,Mack 10 you know you rule hip-hop and Ice Cube...
11246,5V5429MhHn9vSOSZzVKlwK,What I've done - Feast. Remix,James Hersey,"She thinks I can never understand, that we don..."
9916,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,This music does not contain words
4539,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,"Big Syke, Newt, Hank, Beauregard, Big Sur (yea..."
10790,5PbFD3AIyunz5HxocduA9e,"Flava in Ya Ear Remix (feat. Notorious B.I.G.,...",Craig Mack,"Bad Boys, come out and play You know we had to..."


In [31]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features3, normalized_combined_data, k=10)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
1748,1qhgBzRH0jOzoVZJ2K1hDn,Royal Flush (feat. André 3000 & Raekwon),Big Boi,I am the wrong nigga to cross and the first ni...
1598,1ohXrU5DLEfUlfo4tIYbqJ,Where You Belong,Jay Prince,Yeah this is where you belong This is where yo...
7828,4fsyPCAb55yWiyJVbDJLr5,Falling for You,Tamia,HOOK: Could I be falling for you Is this a fan...
13027,6NJ5Q5QujNwwsst5mdSEfF,Die Happy,DREAMERS,"Yeah, I could die happy for a moment Happy 'ca..."
187,13XHuE00ElL5thSxkaEXxK,Only God Can Judge Me,2Pac,Only God can judge me (that right?) Only God c...
14157,74Yw8aWE60rlSAeT49h9BS,Foe Life,Mack 10,Mack 10 you know you rule hip-hop and Ice Cube...
11246,5V5429MhHn9vSOSZzVKlwK,What I've done - Feast. Remix,James Hersey,"She thinks I can never understand, that we don..."
9916,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,This music does not contain words
4539,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,"Big Syke, Newt, Hank, Beauregard, Big Sur (yea..."
10790,5PbFD3AIyunz5HxocduA9e,"Flava in Ya Ear Remix (feat. Notorious B.I.G.,...",Craig Mack,"Bad Boys, come out and play You know we had to..."


## De la clase 

### Query 1

In [51]:
k_nearest_songs = knn_search_simple(scaled_audio_features.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,6EZDbjz2X5XzgIZCtbsrXh,The Passenger,Siouxsie and the Banshees,6.590346,0
1,5VU2hajdO59NNjIz9p9dYD,High So High (feat. Marilyn Rylander),South Park Mexican,6.609109,0
2,37zQVgP3aTLKNvluXLB5Ii,Picture Me Rollin',2Pac,6.707762,0
3,1lJZSsMoWEbrgaY6CxPMf8,Would I Lie to You? - Remastered Version,Eurythmics,6.718950,0
4,5a251qN4R86hro8UZGuO4E,The Streets - Re-Twist,WC,6.728498,0
5,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,6.773228,0
6,1xixUOjgdqMOPZxursTms3,Anything,JAY-Z,6.782510,0
7,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,6.790789,0
8,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,6.813337,0
9,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,6.831569,0


In [50]:
k_nearest_songs = knn_search_simple_priority_queue(scaled_audio_features.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,0zEKOcdfzNkvW1OFnazAQO,sugar honey ice & tea - Edit,Bring Me The Horizon,8.091016,0
1,0ZfM5XfJTtFPhOxAERRnNY,The Look,Roxette,8.332930,0
2,0zG6VemmYlMhAK5MNvBeP7,No Sleep,Bougenvilla,9.576495,0
3,0ZGLuduCPjgWY1n85ykgMe,Back In the Saddle,Aerosmith,9.004776,0
4,0ZgTez1pojM5aCxsV0hrC9,No Guarantee - Remix Version,Chico DeBarge,7.883589,0
5,0Zh5U48tZNeAzzLTV1CVBE,Forgot About Dre,Dr. Dre,8.414665,0
6,0ZhBEJBNEPjHIZdJZ6I1xU,Cojela Que Va Sin Jockey,Daddy Yankee,7.974203,0
7,0zhGp6rH3Em8VTS44hTOmT,One Too Many,FLETCHER,10.143999,0
8,0ZHK2kaWTvSCRWkLflRFgF,Bota pra Tremer,Pedro Sampaio,9.039822,0
9,0zhkLYpmMZyjtv03haC77Y,Antes de Morirme,Miky Woodz,8.233903,0


## Query 2


In [ ]:
k_nearest_songs = knn_search_simple(scaled_audio_features2.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,2xE16muNCmhZwSOv3i1d1H,Mood Swings,A Boogie Wit da Hoodie,14.358522,0
1,5PM96PMKMfD1lLX2lryUsG,Oh Sheila,Ready For The World,15.187100,0
2,3VwZqgfrM3xb1usuLprkTu,Cellophane,FKA twigs,15.342531,0
3,2YIeE3ClUqkonhOViNHbGq,Kiss of Love,Anything Box,15.482506,0
4,6BqWhxll86CGGE6WxgdRqG,Golden Skans,Klaxons,15.527051,0
5,6zMUIb4uce1CzpbjR3vMdN,Foolish,Ashanti,15.560071,0
6,3VFj5odD26z1u2eV6Vwxa6,River,Ibeyi,15.579255,0
7,1IfUGEJiMxAf96sGEgLt31,You Are Everything,Dru Hill,15.648321,0
8,15xWRPHQMeqgdkGzInx3PY,Falling Like The Stars,James Arthur,15.706148,0
9,3aYBjxTMvrEOP0A0UXg9ER,Final Masquerade,Linkin Park,15.708624,0


In [ ]:
k_nearest_songs = knn_search_simple_priority_queue(scaled_audio_features2.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,2xE16muNCmhZwSOv3i1d1H,Mood Swings,A Boogie Wit da Hoodie,14.358522,0
1,5PM96PMKMfD1lLX2lryUsG,Oh Sheila,Ready For The World,15.187100,0
2,3VwZqgfrM3xb1usuLprkTu,Cellophane,FKA twigs,15.342531,0
3,2YIeE3ClUqkonhOViNHbGq,Kiss of Love,Anything Box,15.482506,0
4,6BqWhxll86CGGE6WxgdRqG,Golden Skans,Klaxons,15.527051,0
5,6zMUIb4uce1CzpbjR3vMdN,Foolish,Ashanti,15.560071,0
6,3VFj5odD26z1u2eV6Vwxa6,River,Ibeyi,15.579255,0
7,1IfUGEJiMxAf96sGEgLt31,You Are Everything,Dru Hill,15.648321,0
8,15xWRPHQMeqgdkGzInx3PY,Falling Like The Stars,James Arthur,15.706148,0
9,3aYBjxTMvrEOP0A0UXg9ER,Final Masquerade,Linkin Park,15.708624,0


## Query 3

In [ ]:
k_nearest_songs = knn_search_simple(scaled_audio_features3.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,1qhgBzRH0jOzoVZJ2K1hDn,Royal Flush (feat. André 3000 & Raekwon),Big Boi,7.271711,0
1,1ohXrU5DLEfUlfo4tIYbqJ,Where You Belong,Jay Prince,7.367985,0
2,4fsyPCAb55yWiyJVbDJLr5,Falling for You,Tamia,7.381604,0
3,6NJ5Q5QujNwwsst5mdSEfF,Die Happy,DREAMERS,7.392843,0
4,13XHuE00ElL5thSxkaEXxK,Only God Can Judge Me,2Pac,7.474632,0
5,74Yw8aWE60rlSAeT49h9BS,Foe Life,Mack 10,7.488248,0
6,5V5429MhHn9vSOSZzVKlwK,What I've done - Feast. Remix,James Hersey,7.521663,0
7,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,7.533125,0
8,0bdabO15YOj0iZPg2OujAw,Let The Beat Build,Lil Wayne,7.566567,0
9,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,7.567473,0


In [ ]:
k_nearest_songs = knn_search_simple_priority_queue(scaled_audio_features3.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,1qhgBzRH0jOzoVZJ2K1hDn,Royal Flush (feat. André 3000 & Raekwon),Big Boi,7.271711,0
1,1ohXrU5DLEfUlfo4tIYbqJ,Where You Belong,Jay Prince,7.367985,0
2,4fsyPCAb55yWiyJVbDJLr5,Falling for You,Tamia,7.381604,0
3,6NJ5Q5QujNwwsst5mdSEfF,Die Happy,DREAMERS,7.392843,0
4,13XHuE00ElL5thSxkaEXxK,Only God Can Judge Me,2Pac,7.474632,0
5,74Yw8aWE60rlSAeT49h9BS,Foe Life,Mack 10,7.488248,0
6,5V5429MhHn9vSOSZzVKlwK,What I've done - Feast. Remix,James Hersey,7.521663,0
7,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,7.533125,0
8,0bdabO15YOj0iZPg2OujAw,Let The Beat Build,Lil Wayne,7.566567,0
9,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,7.567473,0


## Range Search
El range search utilizamos el modelo KNN y obtenemos resultados interesantes.

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier


def range_search(query_features=pd.DataFrame(), reference_features=pd.DataFrame(), radius=int()):
    # Solo trabajaremos con el vector característico de las canciones
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)

    # Convertimos el dataframe de la consulta a un array numpy
    query_features_numeric = query_features.to_numpy()

    # Creamos el modelo RadiusNeighborsClassifier
    knn_model = RadiusNeighborsClassifier(radius=radius, metric='euclidean')

    # Entrenamos al modelo con el vector característico de las canciones
    knn_model.fit(reference_features_numeric, reference_features['track_id'])

    # Buscamos vecinos dentro del radio para cada punto de la consulta
    neighbor_indices = knn_model.radius_neighbors(query_features_numeric, return_distance=False)

    # Extraemos información de las canciones encontradas
    song_data = []
    unique_tracks = set()

    for query_index, indices in enumerate(neighbor_indices):
        for neighbor_index in indices:
            track_id = reference_features['track_id'].iloc[neighbor_index]
            if track_id not in unique_tracks:
                unique_tracks.add(track_id)
                song_data.append({
                    'track_id': track_id,
                    'track_name': reference_features['track_name'].iloc[neighbor_index],
                    'track_artist': reference_features['track_artist'].iloc[neighbor_index],
                })

    return song_data

In [ ]:
def range_search_Simple(Q, collection, r):
    result = []

    for _, row in collection.iterrows():
        C = row.iloc[4:].to_numpy()  # Obtener el vector característico desde la quinta columna hasta el final

        # Calcular la distancia entre el objeto de consulta Q y el objeto C en la colección
        dist = distance(Q, C)

        # Verificar si la distancia es menor que el umbral r
        if dist < r:
            result.append({
                'track_id': row['track_id'],
                'track_name': row['track_name'],
                'track_artist': row['track_artist'],
                'distance': dist,
            })

    return result


## Query 1

### Modelo KNN - arbol KD

In [ ]:
radius = 11
indices_within_radius = range_search(scaled_audio_features, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result


/home/mat1u5/Desktop/Ciclo-5/BD2/BD2-Project2/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin
1,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe
2,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green
3,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD
4,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW
...,...,...,...
12490,7K4qs6xArR9izybon4i4gW,Somebody,Erlandsson
12491,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk
12492,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait
12493,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie


### Visto en clase

In [ ]:
radius = 11
indices_within_radius = range_search_Simple(scaled_audio_features.to_numpy(), normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result


,track_id,track_name,track_artist,distance
0,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,2.119183
1,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,8.640200
2,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,8.351174
3,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,10.622183
4,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,8.600338
...,...,...,...,...
12490,7K4qs6xArR9izybon4i4gW,Somebody,Erlandsson,8.728154
12491,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,8.493808
12492,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,7.692296
12493,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,9.247514


## Query 2

### Modelo KNN - arbol KD

In [ ]:
radius = 19
indices_within_radius = range_search(scaled_audio_features2, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/home/mat1u5/Desktop/Ciclo-5/BD2/BD2-Project2/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin
1,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe
2,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green
3,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW
4,00FROhC5g4iJdax5US8jRr,Satisfy You,Diddy
...,...,...,...
11070,7K4qs6xArR9izybon4i4gW,Somebody,Erlandsson
11071,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk
11072,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait
11073,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie


### Visto en Clase

In [ ]:
radius = 19
indices_within_radius = range_search_Simple(scaled_audio_features2.to_numpy(), normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

,track_id,track_name,track_artist,distance
0,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,17.121698
1,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,17.291573
2,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,17.351617
3,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,18.871353
4,00FROhC5g4iJdax5US8jRr,Satisfy You,Diddy,17.231533
...,...,...,...,...
11070,7K4qs6xArR9izybon4i4gW,Somebody,Erlandsson,17.982791
11071,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,17.635905
11072,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,16.134193
11073,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,17.542098


## Query 3

### Modelo KNN - arbol KD

In [ ]:
radius = 12
indices_within_radius = range_search(scaled_audio_features3, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/home/mat1u5/Desktop/Ciclo-5/BD2/BD2-Project2/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin
1,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe
2,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green
3,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD
4,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW
...,...,...,...
12927,7K4qs6xArR9izybon4i4gW,Somebody,Erlandsson
12928,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk
12929,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait
12930,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie


### Visto en clase

In [ ]:
radius = 12
indices_within_radius = range_search_Simple(scaled_audio_features3.to_numpy(), normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

,track_id,track_name,track_artist,distance
0,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,10.180261
1,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,9.345794
2,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,8.884721
3,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,10.116227
4,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,8.868248
...,...,...,...,...
12927,7K4qs6xArR9izybon4i4gW,Somebody,Erlandsson,8.312904
12928,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,8.782484
12929,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,8.766483
12930,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,9.241032


## Aclaraciones

La data frame tiene una cancion pero de distintas versiones y tiempo, cuenta con detalles que otros no tienen.

In [ ]:
# Filtrar el DataFrame original para obtener todas las instancias de la canción con track_id específico
all_instances = songs[songs['track_id'] == '05xbEV5M8DAQmhFKKlHaDd']

# Contar el número de instancias
num_instances = all_instances.shape[0]

# Imprimir el resultado
print(f"La canción con track_id '05xbEV5M8DAQmhFKKlHaDd' se repite {num_instances} veces.")


La canción con track_id '05xbEV5M8DAQmhFKKlHaDd' se repite 1 veces.


In [ ]:
# Identificar posibles versiones duplicadas basadas en ciertas columnas (por ejemplo, artist_name y title)
possible_duplicates = songs[songs.duplicated(['track_name', 'track_album_name'], keep=False)]

# Mostrar el DataFrame con posibles versiones duplicadas
possible_duplicates


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
1,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0.0,0.2160,0.00432,0.007230,0.4890,0.650,111.904,262467.0,en
11,00LfFm08VWeZwB0Zlm24AT,Suga Suga,Baby Bash,"So tight, so fly You got me lifted, you got me...",8.0,2bAkYizbM1rMbOUHpfAV9z,Tha Smokin' Nephew,2003-01-01,90s/00s Hip Hop & RnB,0Ar0Ng9DlAWZtSPBvOQgOa,...,-3.041,0.0,0.2680,0.68800,0.000008,0.0841,0.535,82.331,239027.0,en
31,01Bw3xiZ5PQfn0btMWICaQ,Beautiful Life,Lost Frequencies,"Anywhere you are and you'll go, I follow Mount...",30.0,2WGceQjphDCN6OdXxNBH6x,Beautiful Life,2016-06-03,Tropical Beats,37i9dQZF1DX1bDvyV0IkZ3,...,-6.684,0.0,0.0376,0.12700,0.034700,0.1390,0.339,119.979,161464.0,en
48,023OVLNzXhX0j7CxswUt6D,Cheerleader - Felix Jaehn Remix Radio Edit,OMI,"Kid Ink, egh I think I found a genie in a bott...",67.0,1QwWTLYn9dntAj7l9FRJwY,Me 4 U,2015-10-16,Today's Hits 2000-Present,6a66cg3HcsjYkisYyQcov6,...,-6.081,1.0,0.0305,0.14100,0.000012,0.1380,0.594,118.026,180560.0,en
61,02gIQlGYI0KVK87Uo9Ekjj,I Heard It Through The Grapevine - Single Version,Creedence Clearwater Revival,Bet you're wondering how I knew About your pla...,2.0,4o5NlEZ42fd1GJeKmTA62u,Creedence Covers The Classics,2009-01-01,Classic Hard Rock,1gAeQGQ7hr2q7IYmaejDW6,...,-7.962,1.0,0.0413,0.13300,0.003420,0.0569,0.746,117.626,231760.0,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15339,7x7kYJwduClF3I0ozJpNsZ,Loungin (Who Do Ya Luv) - Remix,LL Cool J,Yeah! Ghetto to ghetto (uh) As we bring the dr...,1.0,3fNwYcwgUi4n1jmQ6ERvxK,Mr. Smith (Deluxe Edition),1995-11-20,"Swingbeat (old skool), New Jack Swing, R&B, Hi...",3krpccUV68nBGAQbvHEZDC,...,-9.839,1.0,0.1810,0.44400,0.004510,0.2050,0.504,99.932,229373.0,en
15356,7xoUc6faLbCqZO6fQEYprd,One Last Time,Ariana Grande,"I was a liar, I gave in to the fire I know I s...",79.0,6EVYTRG1drKdO8OnIQBeEj,My Everything (Deluxe),2014-08-25,Post pop teen,222nc9tKxKhfZ2GBrOpwH3,...,-5.036,1.0,0.0323,0.09300,0.000002,0.0960,0.104,125.026,197267.0,en
15384,7ykaUgkdQWJLsMuOymTV2A,If Not For You,George Harrison,"If not for you Babe, I couldn't even find the ...",59.0,4I4xtHaIFOzhZfp1NIHkY6,All Things Must Pass (Remastered),1970-11-27,I didn’t know perm stood for permanent (wave),3e6gYPyrTbaB8BWgSHCt5j,...,-6.886,1.0,0.0317,0.21900,0.003990,0.0820,0.867,116.119,210720.0,en
15389,7yqU5HODQU91zj8ahUaUX1,I Wanna Dance with Somebody (Who Loves Me),Whitney Houston,Clock strikes upon the hour And the sun begins...,2.0,1wvEC2yY7koRQYebhrj1ZY,Whitney,1987-06-02,80s Pop & Rock Hits and Album Tracks,0XOIK4m26aeYSD61E5nSVW,...,-8.824,1.0,0.0453,0.20700,0.000307,0.0888,0.867,118.814,291293.0,en


In [ ]:
# Seleccionar la canción específica (cambiar 'track_name' y 'track_album_name' según tus necesidades)
cancion_seleccionada = songs[(songs['track_name'] == 'Nobody (feat. Athena Cage)')]

# Contar duplicados para la canción seleccionada
duplicados_cancion_seleccionada = cancion_seleccionada.shape[0]

# Imprimir el resultado
print(f"La canción seleccionada se repite {duplicados_cancion_seleccionada} veces.")

cancion_seleccionada


La canción seleccionada se repite 1 veces.


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
9420,4SF1747p541umnykBp352Q,Nobody (feat. Athena Cage),Keith Sweat,I wanna tease you I wanna please you I wanna s...,62.0,0BzXvdpUKDEk612hLc6rZV,Keith Sweat,1996,"Swingbeat (old skool), New Jack Swing, R&B, Hi...",3krpccUV68nBGAQbvHEZDC,...,-9.449,0.0,0.0318,0.484,0.0,0.0482,0.388,117.907,264773.0,en
